In [2]:
import pandas as pd
import re

In [3]:
file = pd.read_excel('/Users/hrobbin/projects/python/Bosch/JivoChat Report 21-09-02 61305d4165b6d.xlsx', sheet_name=0)

In [4]:
def format_date(raw):
    date = re.search(r"\D(\d{1,}).(\d{1,}).(\d{1,})", raw)
    return date.group(3) + '-' + date.group(2) + '-' + date.group(1)

In [5]:
file.columns = ['x' + str(x) for x in range(13)]
start = file.loc[file['x0'] == 'Сводный отчет по сайтам'].index[0]
ends = file.loc[file['x0'] == 'ИТОГО'].index[0]
file2 = pd.DataFrame(file.iloc[start + 3:ends, :7])
file2.columns = ['site_name', 'dial_acc', 'call_acc', 'dial_not_acc', 'call_not_acc', 'dial_noreply', 'offline_msg']
file2['date'] = format_date(file.iloc[0, 0])
file2.insert(0, 'date', file2.pop('date'))
file2.reset_index(drop=True, inplace=True)

In [13]:
fine_cols = ['date', 'site_name', 'dial_acc', 'call_acc', 'dial_not_acc', 'call_not_acc', 'dial_noreply', 'offline_msg']
values = ""
for ind in file2.index:
    lst = [str(el) for el in file2.iloc[ind].to_list()]
    for i in range(2):
        lst[i] = "\'" + lst[i] + "\'"
    values += "({})".format(", ".join(lst))
    if ind != file2.last_valid_index():
        values+=", "
values = re.sub('Мишлен', 'Michelin', values)
statement = "INSERT IGNORE INTO chats.table_site_reports " + ', '.join(fine_cols) + ") VALUES({})".format(values)
statement

"INSERT IGNORE INTO chats.table_site_reports(date, site_name, dial_acc, call_acc, dial_not_acc, call_not_acc, dial_noreply, offline_msg) VALUES(('2021-09-01', 'tyreplus.ru', 1, 0, 0, 0, 0, 0), ('2021-09-01', '[Yandex] tyreplus.ru', 0, 0, 0, 0, 0, 0), ('2021-09-01', 'michelin.ru', 10, 0, 1, 0, 1, 0), ('2021-09-01', 'VK [Michelin]', 0, 0, 0, 0, 0, 0), ('2021-09-01', 'BFGoodrich.ru', 0, 0, 0, 0, 0, 0), ('2021-09-01', '[Yandex] BFGoodrich', 0, 0, 0, 0, 0, 0), ('2021-09-01', 'Michelin', 1, 0, 0, 0, 0, 1), ('2021-09-01', 'savefuel.michelin.ru', 0, 0, 0, 0, 0, 0), ('2021-09-01', 'b2b experience', 0, 0, 0, 0, 0, 0))"

In [9]:
txt = ""
for ind in file2.index:
    lst = [str(el) for el in file2.iloc[ind].to_list()]
    txt += "({})".format(", ".join(lst))
    if ind != file2.last_valid_index():
        txt+=", "
txt

'(2021-09-01, tyreplus.ru, 1, 0, 0, 0, 0, 0), (2021-09-01, [Yandex] tyreplus.ru, 0, 0, 0, 0, 0, 0), (2021-09-01, michelin.ru, 10, 0, 1, 0, 1, 0), (2021-09-01, VK [Мишлен], 0, 0, 0, 0, 0, 0), (2021-09-01, BFGoodrich.ru, 0, 0, 0, 0, 0, 0), (2021-09-01, [Yandex] BFGoodrich, 0, 0, 0, 0, 0, 0), (2021-09-01, Мишлен, 1, 0, 0, 0, 0, 1), (2021-09-01, savefuel.michelin.ru, 0, 0, 0, 0, 0, 0), (2021-09-01, b2b experience, 0, 0, 0, 0, 0, 0)'

In [53]:
txt = re.sub("Мишлен", "Michelin", txt)
txt = re.sub("[","",txt)
txt = re.sub("]","",txt)
txt

error: unterminated character set at position 0